In [1]:
import os, sys
import collections
from typing import Dict, List
import json
sys.path.append('/home/ubuntu/MLData/work/Repos/NeuralIR/neural_IR')

from utils.metrics import compute_MRR_metrics, compute_recall_at_k


## Load data

In [3]:
with open('/mnt/d/MLData/Repos/neural_IR/assets/msmarco/query_2_groundtruth_passage_small.json', 'r') as file:
    query_2_top_pid_groundtruth = json.load(file)


In [4]:
with open('/mnt/d/MLData/Repos/neural_IR/assets/msmarco/query_2_top_1000_passage_BM25.json', 'r') as file:
    query_2_top_pid_candiates_BM25 = json.load(file)

In [5]:
print(type(query_2_top_pid_groundtruth))
print(type(next(iter(query_2_top_pid_groundtruth))))
print(len(query_2_top_pid_groundtruth))

<class 'dict'>
<class 'str'>
6980


## Re-Ranking result analysis

In [6]:
result_file = "/mnt/d/MLData/Repos/neural_IR/experiments/msmarco_psg_ranking/evaluation/ranking.tsv"

query_2_pid_raw = collections.defaultdict(list)
with open(result_file) as file:
    for line in file:
        parts = line.strip().split()
        query_2_pid_raw[parts[0]].append((parts[2], parts[1]))

query_2_top_pid_candiates = {}
for key in query_2_pid_raw:
    query_2_pid_raw[key].sort()
    query_2_top_pid_candiates[key] = [e[1] for e in query_2_pid_raw[key]]

print(len(query_2_top_pid_candiates))

6836


In [7]:
num_pids = [len(l) for l in query_2_top_pid_candiates.values()]
print(sum(num_pids) // len(num_pids))

109


In [8]:
print(query_2_top_pid_candiates.keys())
print(next(iter(query_2_top_pid_groundtruth.values())))

dict_keys(['188714', '1082792', '995526', '199776', '660957', '820267', '837202', '130825', '408149', '345453', '1019649', '1099065', '542431', '1084910', '959083', '816483', '995825', '559771', '1091246', '167436', '899212', '1101211', '1047854', '991832', '426442', '1093443', '779475', '1040038', '706950', '185299', '574730', '1085008', '1011382', '1062784', '1090701', '973917', '999517', '596716', '289276', '990995', '609628', '988787', '996805', '1025483', '998493', '358455', '435412', '1090730', '348594', '218000', '760512', '665972', '1006751', '596130', '988119', '942221', '1001108', '23285', '1002148', '1097438', '727707', '791140', '259417', '913568', '198246', '370734', '1056265', '432874', '1089312', '141694', '1093781', '1009183', '762059', '931147', '1027650', '170770', '1027817', '939866', '1031456', '480064', '1098608', '234114', '27743', '1089868', '404051', '1075636', '684459', '591940', '313940', '1043545', '510893', '250367', '1101531', '920753', '1054339', '1093552'

### Compute recall



In [7]:
def compute_recall_at_k(query_2_top_pid_groundtruth: Dict[int, List[int]], query_2_top_pid_candiates: Dict[int, List[int]], retrieval_num):
    recall_at_k = [len(set.intersection(set(query_2_top_pid_groundtruth[qid]), set(query_2_top_pid_candiates[qid][:retrieval_num]))) / max(1.0, len(query_2_top_pid_groundtruth[qid]))
                    for qid in query_2_top_pid_candiates]
    recall_at_k = sum(recall_at_k) / len(query_2_top_pid_candiates)
    recall_at_k = round(recall_at_k, 7)
    print(f"Recall @ {retrieval_num} = {recall_at_k}")
    

In [9]:
# reranking performance
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates_BM25, 1000)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates_BM25, 200)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates_BM25, 50)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates_BM25, 10)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates_BM25, 5)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates_BM25, 1)


Recall @ 1000 = 0.8140401
Recall @ 200 = 0.1641476
Recall @ 50 = 0.0337631
Recall @ 10 = 0.0149713
Recall @ 5 = 0.0093123
Recall @ 1 = 0.0022206


In [10]:
# reranking performance
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 1000)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 200)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 50)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 10)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 5)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 1)

# margin 20.0
# Recall @ 1000 = 0.8085809
# Recall @ 200 = 0.3250825
# Recall @ 50 = 0.2128713
# Recall @ 10 = 0.1683168
# Recall @ 5 = 0.1683168
# Recall @ 1 = 0.1485149

# margin 1.0
# Recall @ 1000 = 0.8085809
# Recall @ 200 = 0.2706271
# Recall @ 50 = 0.1633663
# Recall @ 10 = 0.0693069
# Recall @ 5 = 0.0594059
# Recall @ 1 = 0.049505

# margin 15.0 top 10MM
# Recall @ 1000 = 0.8140401
# Recall @ 200 = 0.4320081
# Recall @ 50 = 0.2334408
# Recall @ 10 = 0.1644102
# Recall @ 5 = 0.1577483
# Recall @ 1 = 0.1371896

Recall @ 1000 = 0.8140401
Recall @ 200 = 0.4320081
Recall @ 50 = 0.2334408
Recall @ 10 = 0.1644102
Recall @ 5 = 0.1577483
Recall @ 1 = 0.1371896


### Compute MRR at k

In [11]:
def compute_MRR_metrics(qids_to_relevant_passageids: Dict[int, List[int]], qids_to_ranked_candidate_passages: Dict[int, List[int]], MaxMRRRank:int = 10, verbose: bool=False):
    """Compute MRR metric
    Args:    
    p_qids_to_relevant_passageids (dict): dictionary of query-passage mapping
        Dict as read in with load_reference or load_reference_from_stream
    p_qids_to_ranked_candidate_passages (dict): dictionary of query-passage candidates
    Returns:
        dict: dictionary of metrics {'MRR': <MRR Score>}
    """
    all_scores = {}
    MRR = 0
    qids_with_relevant_passages = 0
    ranking = []
    for qid in qids_to_ranked_candidate_passages:
        if qid in qids_to_relevant_passageids:
            ranking.append(0)
            target_pid = qids_to_relevant_passageids[qid]
            candidate_pid = qids_to_ranked_candidate_passages[qid]
            if len(candidate_pid) < MaxMRRRank and verbose:
                print(f"warning: candidate pid length {len(candidate_pid)} of qid {qid} is smaller than max MRR rank {MaxMRRRank}")
            for i in range(0, min(MaxMRRRank, len(candidate_pid))):
                if candidate_pid[i] in target_pid:
                    MRR += 1/(i + 1)
                    ranking.pop()
                    ranking.append(i+1)
                    break
    if len(ranking) == 0:
        raise IOError("No matching QIDs found. Are you sure you are scoring the evaluation set?")
    
    # average over all queries
    MRR = MRR/len(qids_to_relevant_passageids)
    all_scores[f'MRR @ {MaxMRRRank}'] = MRR
    all_scores['QueriesRanked'] = len(qids_to_ranked_candidate_passages)
    return all_scores

In [12]:
print(compute_MRR_metrics(query_2_top_pid_groundtruth, query_2_top_pid_candiates), 10)

{'MRR @ 10': 0.1519798631009233, 'QueriesRanked': 6980} 10


## Retrieval result analysis

In [26]:
ranking_file = "/home/ubuntu/MLData/work/Repos/NeuralIR/results/MSMARCO-psg/retrieve.py/2022-03-18_22.42.10/ranking.tsv"
query_2_pid_raw = collections.defaultdict(list)
with open(ranking_file) as file:
    for line in file:
        parts = line.strip().split()
        query_2_pid_raw[int(parts[0])].append((int(parts[2]), int(parts[1])))

query_2_top_pid_candiates = {}
for key in query_2_pid_raw:
    query_2_pid_raw[key].sort()
    query_2_top_pid_candiates[key] = [e[1] for e in query_2_pid_raw[key]]

In [27]:
# retrieval performance
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 1000)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 200)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 50)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 10)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 5)
compute_recall_at_k(query_2_top_pid_groundtruth, query_2_top_pid_candiates, 1)

Recall @ 1000 = 0.965
Recall @ 200 = 0.921
Recall @ 50 = 0.827
Recall @ 10 = 0.634
Recall @ 5 = 0.521
Recall @ 1 = 0.231


In [28]:
print(compute_MRR_metrics(query_2_top_pid_groundtruth, query_2_top_pid_candiates), 10)

{'MRR @ 10': 0.36122180151907973, 'QueriesRanked': 6980} 10


## In-depth analysis

### Load textual data